In [0]:
# Connect to ADLS Gen2 using Secret Scope  

service_credential = dbutils.secrets.get(scope="OlympicSecretScope", key="ClientSecretkey")

spark.conf.set("fs.azure.account.auth.type.datalakeolympic.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datalakeolympic.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datalakeolympic.dfs.core.windows.net", "7b5c2be2-27e5-4544-96f5-ed1dbf94be07")
spark.conf.set("fs.azure.account.oauth2.client.secret.datalakeolympic.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datalakeolympic.dfs.core.windows.net", "https://login.microsoftonline.com/8cb8308d-3bb5-4357-9356-8b5723aae493/oauth2/token")

In [0]:
df_athlete = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://raw@datalakeolympic.dfs.core.windows.net/olympic_data/athlete.csv")
df_athlete.createOrReplaceTempView("athleteData")
display(df_athlete)

In [0]:
df_athlete.printSchema()

In [0]:
%sql
SELECT 
initcap(PersonName) AS Athlete, 
Country,
Discipline
FROM
athleteData

In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/athletes")

In [0]:
df_athlete_conformed = spark.read.csv("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/athletes", header=True, inferSchema=True)
display(df_athlete_conformed)

In [0]:
df_coaches = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://raw@datalakeolympic.dfs.core.windows.net/olympic_data/coaches.csv")
df_coaches.createOrReplaceTempView("coachesData")
display(df_coaches)

In [0]:
df_coaches.printSchema()

In [0]:
%sql
SELECT
count(*)
FROM
coachesData

In [0]:
%sql
SELECT 
initcap(Name) AS Coach_Name, 
Country,
Discipline,
Event
FROM
coachesData
WHERE 
Country NOT RLIKE '[^a-zA-Z0-9 ]'


In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/coaches")

In [0]:
%sql
  SELECT *
    FROM coachesData
    WHERE 
    Country RLIKE '[^a-zA-Z0-9 ]'

In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/bad_records")

In [0]:
result_regex = spark.sql("""
    SELECT count(*)
    FROM coachesData
    WHERE 
    Country RLIKE '[^a-zA-Z0-9 ]' 
""")

result_regex.show()

In [0]:
df_coaches_conformed = spark.read.csv("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/coaches", header=True, inferSchema=True)
display(df_coaches_conformed)

In [0]:
df_gender = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://raw@datalakeolympic.dfs.core.windows.net/olympic_data/entriesgender.csv")
df_gender.createOrReplaceTempView("genderData")
display(df_gender)

In [0]:
df_gender.printSchema()

In [0]:
%sql
SELECT 
coalesce(Female,0) AS Number_of_Female,
coalesce(Male,0) AS Number_of_Male,
coalesce(Total,0) AS Total_Participants,
Female/Total AS Avg_Female,
Male/Total AS Avg_Male
FROM genderData

In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/gender")

In [0]:
df_gender_conformed = spark.read.csv("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/gender", header=True, inferSchema=True)
display(df_gender_conformed)

In [0]:
df_medals = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://raw@datalakeolympic.dfs.core.windows.net/olympic_data/medals.csv")
df_medals.createOrReplaceTempView("medalsData")
display(df_medals)

In [0]:
%sql
SELECT
count(*)
FROM
medalsData

In [0]:
df_medals.printSchema()

In [0]:
%sql
SELECT 
Rank,
TeamCountry,
Gold,
Silver,
Bronze,
Total,
`Rank by Total`
FROM medalsData

In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/medals")

In [0]:
df_medals_conformed = spark.read.csv("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/medals", header=True, inferSchema=True)
display(df_medals_conformed)

In [0]:
df_teams = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://raw@datalakeolympic.dfs.core.windows.net/olympic_data/teams.csv")
df_teams.createOrReplaceTempView("teamsData")
display(df_teams)

In [0]:
%sql
SELECT 
TeamName,
Discipline,
Country,
Event
FROM teamsData

In [0]:
_sqldf \
    .write \
    .mode("overwrite") \
    .format("csv") \
    .option("header", "true") \
    .save("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/teams")

In [0]:
df_teams_conformed = spark.read.csv("abfss://conformed@datalakeolympic.dfs.core.windows.net/olympic_data/teams", header=True, inferSchema=True)
display(df_teams_conformed)